In [74]:
import numpy as np
import pandas as pd

In [82]:
n = 20
A = np.array([[1.0/(i+j) if i!=j else 1 for j in range(1, n+1)] for i in range(1, n+1)])
f = np.array([1.0/i for i in range(1, n+1)])

In [83]:
class GaussianElimination:
    def __init__(self, A, f):
        self.__n = len(A)
        self.__A = np.insert(A, self.__n, f, 1)

    def __swap_rows(self, i, j):
        tmp_row = self.__A[i]
        self.__A[i] = self.__A[j]
        self.__A[j] = tmp_row

    def __descent_nulls(self):
        for i in range(self.__n):
            for j in range(i, self.__n):
                if self.__A[j][i] != 0:
                    not_null_index = j
                    break
            self.__swap_rows(i, not_null_index)

    def __forward(self):
         for i in range(self.__n):
            for j in range(i+1, self.__n):
                self.__A[j] = self.__A[j] - self.__A[i] * self.__A[j][i] / self.__A[i][i]
    
    def __backward(self):
        for i in range(self.__n - 1, -1, -1):
            self.__A[i] /= self.__A[i][i]
            for j in range(i + 1, self.__n):
                self.__A[i][self.__n] += -self.__A[i][j] * self.__A[j][self.__n]
                self.__A[i][j] = 0
    
    def solve(self):

        self.__descent_nulls()
        self.__forward()
        self.__backward()

        return np.array([self.__A[i][self.__n] for i in range(self.__n)])

In [84]:
class ZeydelIterations:
    
    def __init__(self, A, f):
        
        n = len(A)

        lower = np.zeros(shape = A.shape)
        diag = np.zeros(shape = A.shape)
        upper = np.zeros(shape = A.shape)

        for i in range(n):
            diag[i][i] = A[i][i]
            for j in range(i + 1, n):
                upper[i][j] = A[i][j]
                lower[n - i - 1][j - i - 1] = A[n - i - 1][j - i - 1]
        
        self.__R = -np.linalg.inv(lower + diag) @ upper
        self.__F = np.linalg.inv(lower + diag) @ f
        self.__x_prev = np.zeros(n)
        self.__x_cur = np.ones(n)
    
    def __prev_cur_difference(self):
        return np.linalg.norm(self.__x_cur - self.__x_prev)

    def __calculate_next(self):
        self.__x_prev = self.__x_cur
        self.__x_cur = self.__R @ self.__x_cur + self.__F

    def solve(self, eps):

        iterations_amount = 0

        while self.__prev_cur_difference() > eps:
            iterations_amount += 1
            self.__calculate_next()

        return {"iterations" : iterations_amount, "solution" : np.copy(self.__x_cur)}

In [85]:
gauss = GaussianElimination(A, f)
zeydel = ZeydelIterations(A, f)

In [86]:
numpy_solution = np.linalg.solve(A,f)
my_gauss_solution = gauss.solve()
zeydel_output = zeydel.solve(1e-4)
my_zeydel_solution = zeydel_output['solution']
zeydel_iterations = zeydel_output['iterations']

In [87]:
columns = ["Numpy solution", "Gauss solution", "Zeydel solution"]
solutions = pd.DataFrame(data = np.array([numpy_solution, my_gauss_solution, my_zeydel_solution]).transpose(), columns=columns)
print(solutions)

    Numpy solution  Gauss solution  Zeydel solution
0         0.920308        0.920308         0.920298
1         0.176812        0.176812         0.176806
2         0.065268        0.065268         0.065265
3         0.028633        0.028633         0.028631
4         0.012783        0.012783         0.012782
5         0.004857        0.004857         0.004857
6         0.000536        0.000536         0.000536
7        -0.001951       -0.001951        -0.001950
8        -0.003426       -0.003426        -0.003425
9        -0.004312       -0.004312        -0.004311
10       -0.004838       -0.004838        -0.004837
11       -0.005138       -0.005138        -0.005137
12       -0.005293       -0.005293        -0.005292
13       -0.005354       -0.005354        -0.005353
14       -0.005351       -0.005351        -0.005350
15       -0.005307       -0.005307        -0.005306
16       -0.005235       -0.005235        -0.005234
17       -0.005145       -0.005145        -0.005145
18       -0.